Import necessary libraries

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, RandomTranslation, RandomRotation, RandomFlip

from keras.utils import to_categorical
import matplotlib.pyplot as plt

from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression

2024-01-16 01:40:22.829116: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-16 01:40:22.830845: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-16 01:40:22.857943: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 01:40:22.857972: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 01:40:22.857990: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
images = np.load('images.npy')
labels = np.load('labels.npy')

encoder = LabelEncoder()
labels = to_categorical(encoder.fit_transform(labels))

NUM_CLASSES = labels.shape[1]

In [8]:

def getCNNModel():
    model = Sequential([
        Input(shape=(64, 64, 1)),
        # RandomTranslation(0.1, 0.1),
        # RandomRotation(0.5),
        # RandomFlip(),
        Conv2D(16, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        BatchNormalization(),   
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        BatchNormalization(), 
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        BatchNormalization(), 
        Flatten(),
        Dense(units=256, activation='relu'),
        BatchNormalization(), 
        Dropout(0.5),
        Dense(units=NUM_CLASSES, activation='softmax')
    ])
    return model

In [3]:
IMG_train, IMG_test, LBL_train, LBL_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [10]:
from keras.optimizers import Adam

k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

best_val_acc = -1
best_model = None

for i, (train_idxs, val_idxs) in enumerate(kfold.split(IMG_train, LBL_train)):
    print(f"Fold {i + 1}/{k}")
    
    model = getCNNModel()
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    train_data = (IMG_train[train_idxs], LBL_train[train_idxs])
    val_data=(IMG_train[val_idxs], LBL_train[val_idxs])

    history = model.fit(*train_data, validation_data=val_data, batch_size=128, epochs=20)
    if history.history['val_accuracy'][-1] > best_val_acc:
        best_val_acc = history.history['val_accuracy'][-1]
        best_model = model

Fold 1/5
Epoch 1/20
365/365 [==============================] - 24s 62ms/step - loss: 4.6060 - accuracy: 0.0802 - val_loss: 3.1397 - val_accuracy: 0.2692
Epoch 2/20
365/365 [==============================] - 24s 67ms/step - loss: 2.9612 - accuracy: 0.2814 - val_loss: 1.8336 - val_accuracy: 0.5859
Epoch 3/20
365/365 [==============================] - 24s 66ms/step - loss: 1.9421 - accuracy: 0.5014 - val_loss: 1.1968 - val_accuracy: 0.7404
Epoch 4/20
365/365 [==============================] - 23s 64ms/step - loss: 1.3882 - accuracy: 0.6377 - val_loss: 0.8641 - val_accuracy: 0.8162
Epoch 5/20
365/365 [==============================] - 24s 66ms/step - loss: 1.0671 - accuracy: 0.7230 - val_loss: 0.6856 - val_accuracy: 0.8537
Epoch 6/20
365/365 [==============================] - 32s 89ms/step - loss: 0.8675 - accuracy: 0.7718 - val_loss: 0.5704 - val_accuracy: 0.8747
Epoch 7/20
365/365 [==============================] - 21s 58ms/step - loss: 0.7242 - accuracy: 0.8106 - val_loss: 0.4813 - val_

KeyboardInterrupt: 

In [11]:
best_model.save('best_cnn_model.h5')


/home/miikey_lol/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
from keras.models import load_model
from keras.models import Model

cnn_model: Sequential = load_model('best_cnn_model.h5')

In [5]:
feature_output = cnn_model.layers[-4].output 
feature_model = Model(inputs=cnn_model.input, outputs=feature_output)

In [6]:
train_features = feature_model.predict(IMG_train)
test_features = feature_model.predict(IMG_test)

457/457 [==============================] - 3s 7ms/step


In [15]:
print(train_features.shape)

(58400, 256)


In [16]:
# Convert from one-hot encoding to integer encoding
LBL_train_int = np.argmax(LBL_train, axis=1)
LBL_test_int = np.argmax(LBL_test, axis=1)

In [17]:
# Convert from integer encoding back to original labels
LBL_train_original = encoder.inverse_transform(LBL_train_int)
LBL_test_original = encoder.inverse_transform(LBL_test_int)

In [ ]:
import numpy as np

# Save train and test features to a single file
np.savez('data.npz',
         train_features=train_features,
         test_features=test_features,
         train_labels=LBL_train_original,
         test_labels=LBL_test_original)

In [ ]:
data = np.load('features.npz')
train_features = data['train_features']
test_features = data['test_features']
train_labels = data['train_labels']
test_labels = data['test_labels']


In [18]:
logistic_regressor = LogisticRegression(solver='lbfgs', max_iter=400)
logistic_regressor.fit(train_features, train_labels)

/home/miikey_lol/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=400)

In [19]:
predictions = logistic_regressor.predict(test_features)

In [20]:
from sklearn.metrics import classification_report

report = classification_report(test_labels, predictions)
print(report)

              precision    recall  f1-score   support

           一       1.00      0.99      1.00       125
           七       0.96      0.96      0.96       138
           上       0.97      0.99      0.98       155
           中       0.95      0.94      0.94       143
           么       0.97      0.95      0.96       150
           九       0.93      0.94      0.94       137
           书       0.95      0.97      0.96       151
           买       0.94      0.96      0.95       149
           了       0.98      0.99      0.98       136
           些       0.95      0.98      0.97       156
           京       0.95      0.94      0.95       149
           人       0.96      0.99      0.98       145
           他       0.96      0.99      0.97       153
           们       0.99      0.97      0.98       131
           会       0.95      0.96      0.96       168
           作       0.93      0.91      0.92       137
           你       0.93      0.93      0.93       137
           候       0.96    

In [21]:
from sklearn.neighbors import KNeighborsClassifier

# Create an instance of the KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)

# Fit the classifier to the training data
knn.fit(train_features, train_labels)

KNeighborsClassifier(n_neighbors=3)

In [22]:
# Predict the labels for the test data
knn_predictions = knn.predict(test_features)

In [23]:
report = classification_report(test_labels, knn_predictions)
print(report)

              precision    recall  f1-score   support

           一       1.00      1.00      1.00       125
           七       0.91      0.97      0.94       138
           上       0.96      0.98      0.97       155
           中       0.92      0.98      0.95       143
           么       0.87      0.97      0.92       150
           九       0.92      0.95      0.94       137
           书       0.96      0.96      0.96       151
           买       0.95      0.94      0.95       149
           了       0.94      0.99      0.96       136
           些       0.93      0.97      0.95       156
           京       0.94      0.95      0.95       149
           人       0.94      0.99      0.96       145
           他       0.95      0.99      0.97       153
           们       0.98      0.98      0.98       131
           会       0.93      0.98      0.96       168
           作       0.88      0.91      0.90       137
           你       0.92      0.94      0.93       137
           候       0.95    

In [24]:
from PIL import Image, ImageOps

def numpy_to_image(numpy_array):
    return Image.fromarray(numpy_array.astype('uint8'))

In [ ]:
import random

# Get the indices of correct and wrong predictions
correct_indices = np.where(predictions == test_labels)[0]
wrong_indices = np.where(predictions != test_labels)[0]

correct_indices = random.sample(list(correct_indices), 4)
wrong_indices = random.sample(list(wrong_indices), 4)

correct_images = [IMG_test[idx] for idx in correct_indices]
wrong_images = [IMG_test[idx] for idx in wrong_indices]

correct_pred = (correct_indices, correct_images)
wrong_pred = (wrong_indices, wrong_images)

# Set the font for the plot
import matplotlib.font_manager as fm
font_path = 'NotoSansCJKsc-Regular.otf'
prop = fm.FontProperties(fname=font_path)

for idxs, images in (wrong_pred, ):
    fig, axs = plt.subplots(2, 2, figsize=(8, 8))
    for i in range(2):
        for j in range(2):
            idx = idxs[2 * i + j]
            numpy_image = images[2 * i + j]
            img = numpy_to_image(numpy_image)
            img = ImageOps.invert(img)
            axs[i][j].imshow(img)
            axs[i][j].set_title(f"Prediction: {predictions[idx]}, Actual: {LBL_test_original[idx]}", fontproperties=prop)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

In [37]:
model = RandomForestClassifier(n_estimators=500, random_state=42)

model.fit(train_features, train_labels)

: 

In [34]:
predictions = model.predict(test_features)

In [35]:
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9328082191780822
